#### Import libarires
import pandas as pd
import numpy as np
import os

In [4]:
# Creata a import path
path = r'C:\Users\SuperHome\Documents\Instacart Basket Analysis\02 Data\Prepared Data'

# Question 1.
Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge
dataframe, which should include your newly derived columns from the previous Exercise.

#### Import dataframe -- customers megered with orders and products

In [5]:
cus_ord = pd.read_pickle(os.path.join(path,'cus_ord_pro.pkl'))

In [6]:
# Checking head
cus_ord.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,...,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range,Busiest_day,busiest_period_of_day
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,2,0,both,Soda,77,7,9.0,Mid-range product,two_busiest days,Most orders
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,1,1,both,Soda,77,7,9.0,Mid-range product,regularly busy,Most orders
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,2,1,both,Soda,77,7,9.0,Mid-range product,two_busiest days,Most orders
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,1,0,both,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,regularly busy,Most orders
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,1,1,both,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,two_slowest days,Most orders


# Question 2.
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [7]:
# Create a subset
df = cus_ord[:1000000]

In [8]:
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,16.025666
2,17.550218
3,17.141279
4,17.725824
5,14.868841
6,17.622972
7,17.594444
8,16.545540


In [10]:
# Using Groupby function fro the entire dataframe
cus_ord.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# Question 3.
Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

### Answer: The subset we created contains 1,000,000 rows, and those rows only include part of the 'department_id'. We shorted some of the 'department_id' compare with the entire dataframe, and also for the mean for every single 'department_id' is different, some of them are greater than the entire dataframe and some of them are less.

# Question 4.
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

#### Step 1. Using transform()function to create a new_column named as 'max_order'

In [12]:
cus_ord['max_order'] = cus_ord.groupby(['user_id'])['order_number'].transform(np.max)

#### Step 2. Using loc() to create the 'loyalty_flag' based on the criteria we were gaven.

In [13]:
cus_ord.loc[cus_ord['max_order'] > 40, 'loyalty_flag'] = 'loyal customer'

In [16]:
cus_ord.loc[(cus_ord['max_order'] <= 40) & (cus_ord['max_order'] > 10), 'loyalty_flag'] = 'regular customer'

In [17]:
cus_ord.loc[cus_ord['max_order'] <= 10, 'loyalty_flag'] = 'new customer'

In [18]:
cus_ord['loyalty_flag'].value_counts()

regular customer    15874128
loyal customer      10282763
new customer         6242841
Name: loyalty_flag, dtype: int64

In [19]:
cus_ord.shape

(32399732, 29)

# Question 5.
The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [20]:
# First, using groupby function to create groups by the 'loyalty_flag'
cus_ord_loy = cus_ord.groupby('loyalty_flag')
print(cus_ord_loy)

In [21]:
# Second, under this created group, check the price whether differ among thses customers.
cus_ord_loy['prices'].mean()

loyalty_flag
loyal customer      7.773575
new customer        7.801206
regular customer    7.798262
Name: prices, dtype: float64

In [22]:
# Or we can group direcly without create new dataframe
cus_ord.groupby('loyalty_flag').agg({'prices':['mean']})

,prices
,mean
loyalty_flag,
loyal customer,7.773575
new customer,7.801206
regular customer,7.798262


## Here,the answer is clear that there is no big difference from the loyalty group.

# Question 6.
The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

#### Step 1. Create a new column named as 'average_price' by using the transform() function

In [23]:
cus_ord['average_price'] = cus_ord.groupby(['user_id'])['prices'].transform(np.mean)

#### Step 2. Create a spending flag based on the new column we created

In [24]:
# Here I will use fro-loop function

spend = []
for value in cus_ord['average_price']:
    if value < 10:
        spend.append('low spender')
    elif value >= 10:
        spend.append('high spender')
    else:
        spend.append('no data')

In [25]:
cus_ord['spending_flag'] = spend

In [26]:
cus_ord.head(2)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,...,aisle_id,department_id,prices,price_range,Busiest_day,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,77,7,9.0,Mid-range product,two_busiest days,Most orders,8,new customer,7.988889,low spender
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,77,7,9.0,Mid-range product,regularly busy,Most orders,8,new customer,7.988889,low spender


# Question 7.
In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

#### Step 1.   Create a new column named as 'median_days_since_prior_orderr' by using the transform() function

In [27]:
cus_ord['median_days_since_prior_order'] = cus_ord.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

#### Step 2. Create a order frequency flag based on the new column

In [29]:
cus_ord.loc[cus_ord['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [30]:
cus_ord.loc[(cus_ord['median_days_since_prior_order'] > 10) & (cus_ord['median_days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [33]:
cus_ord.loc[cus_ord['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [34]:
cus_ord['frequency_flag'].value_counts()

Frequent customer        22787089
Regular customer          6926670
Non-frequent customer     2685973
Name: frequency_flag, dtype: int64

# Export the dataframe as a pickle file

In [35]:
cus_ord.to_pickle(os.path.join(path,'final_cus_ord.pkl'))